In [47]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

## Loan Status Classification

In [48]:
# ppp1 = pd.read_csv('../data/public_up_to_150k_1_230930.csv')
path = '../data/'
dir_list = os.listdir(path)[:-1] # Exclude the data dict
print(dir_list)

['public_up_to_150k_3_230930.csv', 'public_up_to_150k_11_230930.csv', 'cleaned_1and2_reduced.csv', 'public_up_to_150k_6_230930.csv', 'public_up_to_150k_1_230930.csv', 'public_up_to_150k_9_230930.csv', 'public_up_to_150k_4_230930.csv', 'public_up_to_150k_10_230930.csv', 'public_up_to_150k_7_230930.csv', 'public_up_to_150k_2_230930.csv', 'public_150k_plus_230930.csv', '.ipynb_checkpoints', 'placeholder.txt', 'public_up_to_150k_8_230930.csv', 'public_up_to_150k_12_230930.csv', 'public_up_to_150k_5_230930.csv']


In [49]:
loans_1 = pd.read_csv(path + dir_list[0])
loans_2 = pd.read_csv(path + dir_list[1])
ppp = pd.concat([loans_1, loans_2])

In [50]:
ppp.shape

(1800000, 53)

In [51]:
ppp.columns

Index(['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'ProcessingMethod',
       'BorrowerName', 'BorrowerAddress', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'OriginatingLender', 'Originatin

In [52]:
print(ppp['LoanStatus'].value_counts())
print(np.sum(ppp['LoanStatus'].isna())) # no nulls

LoanStatus
Paid in Full    1634401
Charged Off      111965
Exemption 4       53634
Name: count, dtype: int64
0


In [53]:
# Baseline model and class imbalance
paid_frac = np.sum(ppp['LoanStatus'] == 'Paid in Full') / ppp.shape[0]
forgiven_frac = np.sum(ppp['LoanStatus'] == 'Charged Off') / ppp.shape[0]
ex4_frac = np.sum(ppp['LoanStatus'] == 'Exemption 4') / ppp.shape[0]
print(f'Paid in full: {round(paid_frac, 3)}')
print(f'Charged Off: {round(forgiven_frac, 3)}')
print(f'Exemption 4: {round(ex4_frac, 3)}')

Paid in full: 0.908
Charged Off: 0.062
Exemption 4: 0.03


'Paid in full' is the plurality class at 0.908.  Predicting 'paid in full' 90.8% of the time represents the baseline model which a successful classifier will beat.

### Random Forest

In [54]:
# type check
ppp['SBAOfficeCode'] = ppp['SBAOfficeCode'].astype(int)

In [55]:
np.sum(ppp['LoanStatusDate'].isna()) / ppp['LoanStatusDate'].size

0.029796666666666666

We know that about 2.97% of loans are still in arrears, and the LoanStatusDate will be null. We drop this from the model features for now because this will be highly correlated with the target and lead to data leakage.  We can add this feature back in at a later iteration of the model if we want to see whether the date the loan was forgiven or repaid can predict between these two classes only.

In [56]:
np.sum(ppp['UndisbursedAmount'] > 0)

4

In [57]:
ppp[ppp['UndisbursedAmount'] > 0][['UndisbursedAmount', 'CurrentApprovalAmount']]

,UndisbursedAmount,CurrentApprovalAmount
156358,78230.00,78230.00
149720,3000.00,28400.00
772551,0.75,2262.75
809276,330000.00,0.00


Nearly all loans have been disbursed, so we drop UndisbursedAmount from the model because it will not have predictive value.

In [58]:
ppp['RuralUrbanIndicator'].value_counts()

RuralUrbanIndicator
U    1618966
R     181034
Name: count, dtype: int64

In [59]:
np.sum(ppp['RuralUrbanIndicator'].isna())

0

In [60]:
ppp['HubzoneIndicator'].value_counts()

HubzoneIndicator
N    1308199
Y     491801
Name: count, dtype: int64

In [61]:
np.sum(ppp['BusinessAgeDescription'].isna())

0

In [62]:
np.sum(ppp['ProjectCountyName'].isna())

63

We do not know what Project features refer to and there are some missing values. We elect to ignore these features.

In [63]:
np.sum(ppp['CD'].isna())

54

The congressional district associated with a loan (CD) is important for potentially connecting this dataset to related COVID public health data.  There are some missing values, which we will handle as 'unrecorded'.

In [64]:
ppp['CD'] = ppp['CD'].where(ppp['CD'].notna(), 'unrecorded')
np.sum(ppp['CD'].isna())

0

In [65]:
np.sum(ppp['JobsReported'].isna()) # All businesses report number of jobs (business size)

0

In [66]:
ppp['NAICSCode'].dtypes

dtype('float64')

In [67]:
np.sum(ppp['NAICSCode'].isna())

19483

Nearly 20,000 loans do not report the NAICSCode (type of industry) the business operates in.  This information is important for the classifier. We designate unrecorded values as '000000'.

In [68]:
# temp_ppp = ppp

In [69]:
ppp['NAICSCode'] = ppp['NAICSCode'].where(ppp['NAICSCode'].notna(), 0)
np.sum(ppp['NAICSCode'].isna())

0

In [70]:
temp = ppp['NAICSCode'].astype(int).astype(str).replace(to_replace='0', value='000000')
np.sum(temp == '000000')

19483

In [71]:
ppp['NAICSCode'] = temp

In [72]:
ppp['NAICSCode'].dtypes

dtype('O')

In [73]:
np.sum(ppp[['Race', 'Ethnicity', 'Gender', 'Veteran']].isna(), axis=0)

Race         0
Ethnicity    0
Gender       0
Veteran      0
dtype: int64

Demographic information of the business owner has no nulls.

In [74]:
np.sum(ppp[['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED']].isna(), axis=0)

UTILITIES_PROCEED            1279767
PAYROLL_PROCEED                 1379
MORTGAGE_INTEREST_PROCEED    1765119
RENT_PROCEED                 1730784
REFINANCE_EIDL_PROCEED       1784389
HEALTH_CARE_PROCEED          1779194
DEBT_INTEREST_PROCEED        1784319
dtype: int64

In [75]:
# fill nulls as zero
temp = ppp[['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED']]

temp = temp.replace(to_replace=np.nan, value=0.0)
np.sum(temp.isna(), axis=0)

# df = ppp.drop(['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED']).merge(temp, how='left')
# df.head(10)

UTILITIES_PROCEED            0
PAYROLL_PROCEED              0
MORTGAGE_INTEREST_PROCEED    0
RENT_PROCEED                 0
REFINANCE_EIDL_PROCEED       0
HEALTH_CARE_PROCEED          0
DEBT_INTEREST_PROCEED        0
dtype: int64

In [76]:
temp_df = ppp.drop(columns=['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED'])

In [77]:
temp_df['UTILITIES_PROCEED'] = temp['UTILITIES_PROCEED']

In [78]:
# temp_df.shape
# temp_df['PAYROLL_PROCEED'] = temp['PAYROLL_PROCEED']
# temp_df.shape

for proceed in ['UTILITIES_PROCEED', 'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED']:
    ppp[proceed] = temp[proceed]

ppp.shape

(1800000, 53)

I brute forced adding these columns back because the joins were just doubling the rows instead of using the shared index.

In [79]:
np.sum(ppp[['BusinessType', 'OriginatingLenderLocationID']].isna(), axis=0)

BusinessType                   163
OriginatingLenderLocationID      0
dtype: int64

In [80]:
# Handle 163 null business type
bt_temp = ppp['BusinessType'].replace(to_replace=np.nan, value='unrecorded')
np.sum(bt_temp.isna())

ppp['BusinessType'] = bt_temp

In [81]:
ppp['OriginatingLenderLocationID'] = ppp['OriginatingLenderLocationID'].astype(int).astype(str)

In [82]:
# Choose features we believe will be most predictive
drop_features = ['LoanNumber', 'DateApproved', 'BorrowerName', 'BorrowerAddress', 'LoanStatusDate', 'UndisbursedAmount', 'ServicingLenderName', 'ServicingLenderAddress', 'ProjectCity', 'ProjectCountyName', 'ProjectState', 'ProjectZip', 'OriginatingLender', 'OriginatingLenderCity', 'OriginatingLenderState', 'ForgivenessAmount', 'ForgivenessDate']
temp_dropped = ppp.drop(columns=drop_features)

In [83]:
temp_dropped.columns

Index(['SBAOfficeCode', 'ProcessingMethod', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatus', 'Term', 'SBAGuarantyPercentage',
       'InitialApprovalAmount', 'CurrentApprovalAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderCity',
       'ServicingLenderState', 'ServicingLenderZip', 'RuralUrbanIndicator',
       'HubzoneIndicator', 'LMIIndicator', 'BusinessAgeDescription', 'CD',
       'JobsReported', 'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'Gender', 'Veteran', 'NonProfit'],
      dtype='object')

In [84]:
ppp = temp_dropped

In [85]:
ppp.columns

Index(['SBAOfficeCode', 'ProcessingMethod', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatus', 'Term', 'SBAGuarantyPercentage',
       'InitialApprovalAmount', 'CurrentApprovalAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderCity',
       'ServicingLenderState', 'ServicingLenderZip', 'RuralUrbanIndicator',
       'HubzoneIndicator', 'LMIIndicator', 'BusinessAgeDescription', 'CD',
       'JobsReported', 'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'Gender', 'Veteran', 'NonProfit'],
      dtype='object')


While these proceed data are very important, we have to decide how to best use these.  For now, we can use all of these features as-is.  We must however fill the null values as $0.00.

We fit a model to the reduced df.

In [86]:
np.sum(ppp.isna(), axis=0)

SBAOfficeCode                        0
ProcessingMethod                     0
BorrowerCity                        16
BorrowerState                        0
BorrowerZip                         13
LoanStatus                           0
Term                                 0
SBAGuarantyPercentage                0
InitialApprovalAmount                0
CurrentApprovalAmount                0
FranchiseName                  1780395
ServicingLenderLocationID            0
ServicingLenderCity                  0
ServicingLenderState                 0
ServicingLenderZip                   0
RuralUrbanIndicator                  0
HubzoneIndicator                     0
LMIIndicator                         0
BusinessAgeDescription               0
CD                                   0
JobsReported                         0
NAICSCode                            0
Race                                 0
Ethnicity                            0
UTILITIES_PROCEED                    0
PAYROLL_PROCEED          

In [87]:
ppp = ppp.drop(columns=['FranchiseName', 'NonProfit'])
np.sum(ppp.isna(), axis=0)

SBAOfficeCode                   0
ProcessingMethod                0
BorrowerCity                   16
BorrowerState                   0
BorrowerZip                    13
LoanStatus                      0
Term                            0
SBAGuarantyPercentage           0
InitialApprovalAmount           0
CurrentApprovalAmount           0
ServicingLenderLocationID       0
ServicingLenderCity             0
ServicingLenderState            0
ServicingLenderZip              0
RuralUrbanIndicator             0
HubzoneIndicator                0
LMIIndicator                    0
BusinessAgeDescription          0
CD                              0
JobsReported                    0
NAICSCode                       0
Race                            0
Ethnicity                       0
UTILITIES_PROCEED               0
PAYROLL_PROCEED                 0
MORTGAGE_INTEREST_PROCEED       0
RENT_PROCEED                    0
REFINANCE_EIDL_PROCEED          0
HEALTH_CARE_PROCEED             0
DEBT_INTEREST_

In [88]:
# drop rows with null borrower city and zip

temp = ppp[ppp['BorrowerCity'].notna()]
ppp = temp
np.sum(ppp.isna(), axis=0)

SBAOfficeCode                  0
ProcessingMethod               0
BorrowerCity                   0
BorrowerState                  0
BorrowerZip                    0
LoanStatus                     0
Term                           0
SBAGuarantyPercentage          0
InitialApprovalAmount          0
CurrentApprovalAmount          0
ServicingLenderLocationID      0
ServicingLenderCity            0
ServicingLenderState           0
ServicingLenderZip             0
RuralUrbanIndicator            0
HubzoneIndicator               0
LMIIndicator                   0
BusinessAgeDescription         0
CD                             0
JobsReported                   0
NAICSCode                      0
Race                           0
Ethnicity                      0
UTILITIES_PROCEED              0
PAYROLL_PROCEED                0
MORTGAGE_INTEREST_PROCEED      0
RENT_PROCEED                   0
REFINANCE_EIDL_PROCEED         0
HEALTH_CARE_PROCEED            0
DEBT_INTEREST_PROCEED          0
BusinessTy

Some categorical parameters have too many unique values and are also not very informative.  For example BorrowerCity, Address, Zip.
We will drop these categories as they are low priority for inclusion.

In [89]:
low_priority_cats = ['BorrowerCity', 'BorrowerState', 'ServicingLenderCity', 'ServicingLenderCity', 'ServicingLenderState', 'ServicingLenderZip', ] # 'BorrowerZip' added back for 09 model bc it is handling current feature size ok, and it may be valuable for econogeo info
ppp = ppp.drop(columns=low_priority_cats)

## Binarize the target variable to transform multi-class to binary classification
This will greatly simplify fitting and interpreting the metrics.  It also suits the premise of the business recommendations because we are interested in reducing the burden poorly allocated PPP funds, which could be redirected to other kinds of economic relief.

In [90]:
ppp['LoanStatus'] = ppp['LoanStatus'].replace({'Paid in Full' : 1, 'Charged Off' : 0, 'Exemption 4' : 0})

ppp['LoanStatus'].value_counts(normalize=True)

LoanStatus
1    0.908
0    0.092
Name: proportion, dtype: float64

The nature of our goal:


1 - Paid in full
0 - Defaulted

It is most important to detect loans which will be defaulted on (0). 
It is most deleterious to detect false positives. (predicting paid (1) when it is actually defaulted (0)). because that means we gave a loan which we thought would be recouped but we actually lost the money. We must maximize *precision* to prevent risk.

False negative is when we predict the loan will not be recouped (0) but we actually get the money back (1).  This is less costly because we don't lose any money directly. Although there are indirect consequences like failing to support a business that could have been saved (the opportunity cost of failing to stimulate the economy).


## Here
A reduced, cleaned df that is ready to be fit to a rf model.

In [91]:
ppp.to_csv('../data/cleaned_1and2_reduced.csv', index=False)